# A Decision Tree Classifier in Julia
#### Ron Rounsifer | cs678 | GVSU

In [122]:
using DataFrames # hold data
using CSV # read data
using DataStructures # collection (counter)
using CPUTime # times code
using Profile # profiles code
using Distributed # for parallel processing
setprecision(300) # set the decimal precision to 300 bits

300

## Main Method 
#### (CPU time - 0.004 avg on 2 threads)
1. handles the control flow

In [129]:
struct Node{T<:String}
    class::T
end

function main()
    
    # all the data and info
    df = read_data()::DataFrame
    N = size(df)[1]::Int64
    num_classes = length(unique(@view df[:classification]))::Int64
    
    # Case 1:
    # if all of the conditions are the same, return that value
    if num_classes == 1
        return Node(df[1,:classification]::String)
    end
    
    #Case 2:
    # if there are no attributes, return the most common occurrence
    if (length(@view df[1,:])-1) == 0
        return Node(string(findmax(counter(df[:classification]))))
    end
    
    # Case 3:
    # if the data is normal then the root of the tree must be found
    root = find_root(df, N, num_classes)
end

main (generic function with 1 method)

## Read Data 
Loads the data and checks if there are multiple unique values in the classification column.

* <b>Case 1:</b> 1 classification, returns a Leaf of the only class.

* <b>Case 2:</b> 2+ classes and 0 attributes, return a Leaf of the most common class.

* <b>Case</b> 3: Normal data. Returns a dataframe


In [78]:
function read_data()
    df = CSV.read("./data/training.txt", delim=',', header=1)
    output_col = similar(df[:classification], nrow(df))
    return df
end

read_data (generic function with 1 method)

## Find Root
When a valid dataset is entered, the root of the tree must be determined. 

This class iterates through each class and determines which provides the tree with the most information gain. 

A node is created for this class and returned.

In [190]:
function find_root(df::DataFrame, N::Int64, n_classes::Int64)

    # array to hold class entropies.
    entropies = Vector{Float64}()
    
    # calculates entropy of each class
    for c in counter(@view df[:classification])
        print(c[1])
        print('\n')
        entropies[c[1]] = -(calc_entropy(c[2], N))
    end
    
    # calculates attribute with most information gain
    
    print(entropies)
    return
end

find_root (generic function with 2 methods)

In [191]:
function calc_entropy(in_class::Int64, N::Int64)
    p = (in_class/N)::Float64
    
    return (p * log(2,p))
end

calc_entropy (generic function with 2 methods)

### Code Analysis
#### Timing

In [192]:
@time @CPUtime main()

poor
vgood
good
acceptable
Dict("poor"=>0.352778,"vgood"=>0.174015,"good"=>0.154445,"acceptable"=>0.484143)elapsed CPU time: 0.193314 seconds
  0.253620 seconds (295.45 k allocations: 14.063 MiB, 8.19% gc time)


#### Profiling

In [120]:
@profile main()
Profile.print(format=:flat)

 Count File                        Line Function                               
   141 ./In[69]                       2 read_data()                            
   141 ./In[78]                      12 main()                                 
     2 ./abstractarray.jl           946 _getindex                              
     1 ./abstractarray.jl            57 axes                                   
    12 ./abstractarray.jl            75 axes                                   
    11 ./abstractarray.jl            93 axes1                                  
     4 ./abstractarray.jl           434 checkbounds                            
     4 ./abstractarray.jl           449 checkbounds                            
     3 ./abstractarray.jl           845 copymutable                            
     2 ./abstractarray.jl           668 copyto!(::Array{Int64,1}, ::Core.Com...
     3 ./abstractarray.jl           672 copyto!(::Array{Int64,1}, ::Core.Com...
     5 ./abstractarray.jl           745 